## Baseline, randomly guessing that 20% of the data is popular

In [162]:
import random
my_list = [True for x in range(2)] +  [False for x in range(8)]
accuracy_baseline = np.zeros(shape=(1,9))
recall_baseline = np.zeros(shape=(1,9))
precision_baseline = np.zeros(shape=(1,9))
f1_baseline = np.zeros(shape=(1,9))

In [163]:
df_sf_temp = df_sf_2017.copy()
start_month = 1
end_month = 4

while end_month <13:
    y_test = df_sf_temp[df_sf_temp['month'] == end_month]['popular']
    y_pred = pd.Series(random.choice(my_list) for x in range(y_test.size))
    accuracy_baseline[0][start_month-1] = accuracy_score(y_test, y_pred)
    recall_baseline[0][start_month-1] = recall_score(y_test, y_pred)
    precision_baseline[0][start_month-1] = precision_score(y_test, y_pred)
    f1_baseline[0][start_month-1] = f1_score(y_test, y_pred)
    start_month += 1
    end_month += 1

print(accuracy_baseline.mean())
print(recall_baseline.mean()) 
print(precision_baseline.mean())
print(f1_baseline.mean())


0.6760991542390486
0.19911653217717348
0.2076371303722557
0.20267624173813048


In [32]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime as dt
from sklearn.ensemble import RandomForestClassifier

pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [62]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [86]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Load pickle data

In [33]:
df_sf_2017 = pickle.load(open('../data_sf_2017.p', 'rb'))

In [34]:
len(df_sf_2017)

103956

# NLP

In [22]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import time
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

In [23]:
df_sf_2017.description.fillna(value='None', inplace=True)

In [24]:
%%time

def process_text(text):
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    stemmed = []
    snowball = SnowballStemmer("english")
    for item in tokens:
        stemmed.append(snowball.stem(item))
        
    lemmatized = []
    wordnet = WordNetLemmatizer()
    for item in stemmed:
        lemmatized.append(wordnet.lemmatize(item))
    
    return lemmatized
    

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.4 µs


## split data - running NLP on description column

In [8]:
def split_data(start_month, end_month):
    df_X_train = df_sf_2017[(df_sf_2017['month'] >= start_month) & (df_sf_2017['month'] < end_month)]['description']
    y_train = df_sf_2017[(df_sf_2017['month'] >= start_month) & (df_sf_2017['month'] < end_month)]['popular']

    df_X_test = df_sf_2017[df_sf_2017['month'] == end_month]['description']
    y_test = df_sf_2017[df_sf_2017['month'] == end_month]['popular']
    
    return df_X_train, y_train, df_X_test, y_test

## Run CountVectorizer

In [9]:
%%time
def run_tf_vec(df_X_train, df_X_test):
    tf_vectorizer_train = CountVectorizer(analyzer=process_text).fit(df_X_train)
    X_train = tf_vectorizer_train.transform(df_X_train)
    tf_vectorizer_test = CountVectorizer(analyzer=process_text, vocabulary=tf_vectorizer_train.vocabulary_).fit(df_X_test)
    X_test = tf_vectorizer_test.transform(df_X_test)
    return X_train, X_test

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.54 µs


In [44]:
def predict_tf_nb(X_train, y_train, X_test, y_test):
    from sklearn.naive_bayes import MultinomialNB
    nb = MultinomialNB()
    nb.fit(X_train, y_train)
    pickle.dump(nb, open('nb_model'+ str(model_num) + '.p', 'wb'))
    preds = nb.predict(X_test)
    scores_tf_nb[0][model_num] = accuracy_score(y_test, preds)
    scores_tf_nb[1][model_num] = recall_score(y_test, nb.predict(X_test))
    scores_tf_nb[2][model_num] = precision_score(y_test, nb.predict(X_test))
    scores_tf_nb[3][model_num] = f1_score(y_test, nb.predict(X_test))
    return scores_tf_nb

## With the CountVectorizer, run with RandomForest 

In [46]:
def predict_tf_rf(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators = 10, n_jobs = -1, random_state=0, class_weight = {0:.95, 1:.05})
    rf.fit(X_train, y_train)
    predicted = rf.predict(X_test)
    pickle.dump(rf, open('rf_nlp_countvec_50' + str(model_num) + '.p', 'wb'))
    scores_tf_rf[0][model_num] = accuracy_score(y_test, predicted)
    scores_tf_rf[1][model_num] = recall_score(y_test, rf.predict(X_test))
    scores_tf_rf[2][model_num] = precision_score(y_test, rf.predict(X_test))
    scores_tf_rf[3][model_num] = f1_score(y_test, rf.predict(X_test))
    return scores_tf_rf

In [42]:
scores_tf_nb = np.zeros(shape=(4,9))
scores_tf_rf = np.zeros(shape=(4,9))

model_num = 0
start_month = 1
end_month = 4

In [47]:
%%time
while end_month <13:
    df_X_train, y_train, df_X_test, y_test = split_data(start_month, end_month)
    X_train, X_test = run_tf_vec(df_X_train, df_X_test)
    scores_tf_nb = predict_tf_nb(X_train, y_train, X_test, y_test)
    scores_tf_rf = predict_tf_rf(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    model_num += 1
    start_month += 1
    end_month += 1
    print('tf_nb')
    print(scores_tf_nb)
    print('tf_rf')
    print(scores_tf_rf)

1 4 0
tf_nb
[[0.85448276 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.66372796 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.59014558 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.62477771 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
tf_rf
[[0.95551724 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.80163728 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.9464684  0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.86805319 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
2 5 1
tf_nb
[[0.85448276 0.85868071 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.66372796 0.67674858 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.5901

In [14]:
accuracy_2017_tf_nb = np.mean(scores_tf_nb[0])
recall_2017_tf_nb = np.mean(scores_tf_nb[1])
precision_2017_tf_nb = np.mean(scores_tf_nb[2])
f1_score_2017_tf_nb = np.mean(scores_tf_nb[3])
print(accuracy_2017_tf_nb)
print(recall_2017_tf_nb)
print(precision_2017_tf_nb)
print(f1_score_2017_tf_nb)

0.8455697882468312
0.6745968354038901
0.6159944961116426
0.6435671085940798


In [15]:
accuracy_2017_tf_rf = np.mean(scores_tf_rf[0])
recall_2017_tf_rf = np.mean(scores_tf_rf[1])
precision_2017_tf_rf = np.mean(scores_tf_rf[2])
f1_score_2017_tf_rf = np.mean(scores_tf_rf[3])
print(accuracy_2017_tf_rf)
print(recall_2017_tf_rf)
print(precision_2017_tf_rf)
print(f1_score_2017_tf_rf)

0.946939377339736
0.7846092062627434
0.9502124982965571
0.8590649087408558


## Try running with TF-IDF

In [25]:
%%time
def run_tf_idf_vec(df_X_train, df_X_test):
    tf_idf_vectorizer_train = TfidfVectorizer(analyzer=process_text)
    X_train2 = tf_idf_vectorizer_train.fit_transform(df_X_train).toarray()
    tf_idf_vectorizer_test = TfidfVectorizer(analyzer=process_text, vocabulary=tf_idf_vectorizer_train.vocabulary_)
    X_test2 = tf_idf_vectorizer_test.fit_transform(df_X_test).toarray()
    return X_train2, X_test2

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.39 µs


In [26]:
def predict_tf_idf_nb(X_train2, y_train, X_test2, y_test):
    from sklearn.naive_bayes import GaussianNB
    nb = GaussianNB()
    nb.fit(X_train2, y_train)
    # pickle.dump(nb, open('nb_model_guassian.p', 'wb'))
    preds = nb.predict(X_test2)
    scores_tf_idf_nb[0][model_num] = accuracy_score(y_test, preds)
    scores_tf_idf_nb[1][model_num] = recall_score(y_test, nb.predict(X_test2))
    scores_tf_idf_nb[2][model_num] = precision_score(y_test, nb.predict(X_test2))
    scores_tf_idf_nb[3][model_num] = f1_score(y_test, nb.predict(X_test2))
    return scores_tf_idf_nb

## With the TF-IDF, run with RandomForest 

In [27]:
def predict_tf_idf_rf(X_train2, y_train, X_test2, y_test):
    rf = RandomForestClassifier(n_estimators = 10, n_jobs=-1, random_state=0, class_weight = {0:.95, 1:.05})
    rf.fit(X_train2, y_train)
    predicted = rf.predict(X_test2)
#     pickle.dump(rf, open('rf_nlp_50.p', 'wb'))
    scores_tf_idf_rf[0][model_num] = accuracy_score(y_test, predicted)
    scores_tf_idf_rf[1][model_num] = recall_score(y_test, rf.predict(X_test2))
    scores_tf_idf_rf[2][model_num] = precision_score(y_test, rf.predict(X_test2))
    scores_tf_idf_rf[3][model_num] = f1_score(y_test, rf.predict(X_test2))
    return scores_tf_idf_rf

In [28]:
scores_tf_idf_nb = np.zeros(shape=(4,9))
scores_tf_idf_rf = np.zeros(shape=(4,9))

model_num = 0
start_month = 1
end_month = 4

In [29]:
while end_month <13:
    df_X_train, y_train, df_X_test, y_test = split_data(start_month, end_month) 
    X_train2, X_test2 = run_tf_idf_vec(df_X_train, df_X_test)
    scores_tf_idf_nb = predict_tf_idf_nb(X_train2, y_train, X_test2, y_test)
    scores_tf_idf_rf = predict_tf_idf_rf(X_train2, y_train, X_test2, y_test)
    print(start_month, end_month, model_num)
    model_num += 1
    start_month += 1
    end_month += 1
    print('tf_idf_nb')
    print(scores_tf_idf_nb)
    print('tf_idf_rf')
    print(scores_tf_idf_rf)

1 4 0
tf_idf_nb
[[0.61356322 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.93702771 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.31326316 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.46954875 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
tf_idf_rf
[[0.94597701 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.75692695 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.93468118 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.83646486 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
2 5 1
tf_idf_nb
[[0.61356322 0.61108566 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.93702771 0.93194707 0.         0.         0.         0.
  0.         0.         0.      

In [30]:
accuracy_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[0])
recall_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[1])
precision_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[2])
f1_score_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[3])
print(accuracy_2017_tf_idf_nb)
print(recall_2017_tf_idf_nb)
print(precision_2017_tf_idf_nb)
print(f1_score_2017_tf_idf_nb)

0.6223984688066541
0.9360869500755025
0.3467272958014258
0.5052056786500755


In [31]:
accuracy_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[0])
recall_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[1])
precision_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[2])
f1_score_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[3])
print(accuracy_2017_tf_idf_rf)
print(recall_2017_tf_idf_rf)
print(precision_2017_tf_idf_rf)
print(f1_score_2017_tf_idf_rf)

0.9383264864764923
0.7493591861607887
0.9398408080684738
0.8334356589996533


## Kmeans

In [164]:
from sklearn.cluster import KMeans
from collections import Counter
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram

In [ ]:
features = vectorizer.get_feature_names()
kmeans = KMeans()
kmeans.fit(X_train)

In [ ]:
print("\n2) cluster centers:")
print(kmeans.cluster_centers_)


In [ ]:
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
print("\n3) top features (words) for each cluster:")
for num, centroid in enumerate(top_centroids):
    print("%d: %s" % (num, ", ".join(features[i] for i in centroid)))
